<a href="https://colab.research.google.com/github/kapeeshvarma/Book-Recommendation-System/blob/main/Book_Recommendation_System_Capstone_Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#**BOOK RECOMMENDATION SYSTEM**

Recommender systems are algorithms aimed at suggesting relevant items to users (movies, books, products). Recommender systems are really critical in some industries as they can generate a huge amount of income when they are efficient or also be a way to stand out significantly from competitors.

In this project, we create a book recommendation system for users through Unsupervised Machine Learning using three datasets - Books.csv, Users.csv, Ratings.csv

##Importing Data and Modules

In [1]:
# Import the necessary modules
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import warnings
warnings.filterwarnings("ignore")

In [2]:
# Read in the datasets
data = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Book Recommendation System - Kapeesh Varma/Books.csv')
users = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Book Recommendation System - Kapeesh Varma/Users.csv')
ratings = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Book Recommendation System - Kapeesh Varma/Ratings.csv')

##Exploratory Data Analysis and Visualizations

In [3]:
# Print the head of the Books DataFrame
data = data.iloc[:100000,:]
data.head()

,ISBN,Book-Title,Book-Author,Year-Of-Publication,Publisher,Image-URL-S,Image-URL-M,Image-URL-L
0,0195153448,Classical Mythology,Mark P. O. Morford,2002,Oxford University Press,http://images.amazon.com/images/P/0195153448.0...,http://images.amazon.com/images/P/0195153448.0...,http://images.amazon.com/images/P/0195153448.0...
1,0002005018,Clara Callan,Richard Bruce Wright,2001,HarperFlamingo Canada,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...
2,0060973129,Decision in Normandy,Carlo D'Este,1991,HarperPerennial,http://images.amazon.com/images/P/0060973129.0...,http://images.amazon.com/images/P/0060973129.0...,http://images.amazon.com/images/P/0060973129.0...
3,0374157065,Flu: The Story of the Great Influenza Pandemic...,Gina Bari Kolata,1999,Farrar Straus Giroux,http://images.amazon.com/images/P/0374157065.0...,http://images.amazon.com/images/P/0374157065.0...,http://images.amazon.com/images/P/0374157065.0...
4,0393045218,The Mummies of Urumchi,E. J. W. Barber,1999,W. W. Norton &amp; Company,http://images.amazon.com/images/P/0393045218.0...,http://images.amazon.com/images/P/0393045218.0...,http://images.amazon.com/images/P/0393045218.0...


In [4]:
# Print a concise summary of the Books DataFrame
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 8 columns):
 #   Column               Non-Null Count   Dtype 
---  ------               --------------   ----- 
 0   ISBN                 100000 non-null  object
 1   Book-Title           100000 non-null  object
 2   Book-Author          100000 non-null  object
 3   Year-Of-Publication  100000 non-null  object
 4   Publisher            100000 non-null  object
 5   Image-URL-S          100000 non-null  object
 6   Image-URL-M          100000 non-null  object
 7   Image-URL-L          100000 non-null  object
dtypes: object(8)
memory usage: 6.1+ MB


In [5]:
# Print the head of the Users DataFrame
users = users.iloc[:100000,:]
users.head()

,User-ID,Location,Age
0,1,"nyc, new york, usa",NaN
1,2,"stockton, california, usa",18.0
2,3,"moscow, yukon territory, russia",NaN
3,4,"porto, v.n.gaia, portugal",17.0
4,5,"farnborough, hants, united kingdom",NaN


In [6]:
# Print a concise summary of the Users DataFrame
users.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 3 columns):
 #   Column    Non-Null Count   Dtype  
---  ------    --------------   -----  
 0   User-ID   100000 non-null  int64  
 1   Location  100000 non-null  object 
 2   Age       60269 non-null   float64
dtypes: float64(1), int64(1), object(1)
memory usage: 2.3+ MB


In [7]:
# Print the head of the Ratings DataFrame
ratings = ratings.iloc[:100000,:]
ratings.head()

,User-ID,ISBN,Book-Rating
0,276725,034545104X,0
1,276726,0155061224,5
2,276727,0446520802,0
3,276729,052165615X,3
4,276729,0521795028,6


In [8]:
# Print a concise summary of the Books DataFrame
ratings.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 3 columns):
 #   Column       Non-Null Count   Dtype 
---  ------       --------------   ----- 
 0   User-ID      100000 non-null  int64 
 1   ISBN         100000 non-null  object
 2   Book-Rating  100000 non-null  int64 
dtypes: int64(2), object(1)
memory usage: 2.3+ MB


In [9]:
# Merge Books and Ratings dataframes
data = pd.merge(data,ratings,on='ISBN')

In [10]:
# Reindex and rename the columns of the DataFrame
data = data.reindex(columns=["ISBN", "User-ID", "Book-Title", "Book-Author", "Year-Of-Publication", "Publisher", "Book-Rating", "Image-URL-S", "Image-URL-M", "Image-URL-L"])
data.rename(columns={"User-ID" : "user_id", 'Book-Title' : 'Title', 'Book-Author' : 'Author', "Book-Rating" : "Rating"}, inplace=True)

In [11]:
# Print the head of the new dataframe
data = data.iloc[:100000,:]
data.head()

,ISBN,user_id,Title,Author,Year-Of-Publication,Publisher,Rating,Image-URL-S,Image-URL-M,Image-URL-L
0,0195153448,2,Classical Mythology,Mark P. O. Morford,2002,Oxford University Press,0,http://images.amazon.com/images/P/0195153448.0...,http://images.amazon.com/images/P/0195153448.0...,http://images.amazon.com/images/P/0195153448.0...
1,0002005018,8,Clara Callan,Richard Bruce Wright,2001,HarperFlamingo Canada,5,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...
2,0002005018,11400,Clara Callan,Richard Bruce Wright,2001,HarperFlamingo Canada,0,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...
3,0002005018,11676,Clara Callan,Richard Bruce Wright,2001,HarperFlamingo Canada,8,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...
4,0060973129,8,Decision in Normandy,Carlo D'Este,1991,HarperPerennial,0,http://images.amazon.com/images/P/0060973129.0...,http://images.amazon.com/images/P/0060973129.0...,http://images.amazon.com/images/P/0060973129.0...


In [12]:
# Visualize the mean rating count of the books data
rating_cnt = pd.DataFrame(data.groupby('Title').mean()['Rating'])
rating_cnt = rating_cnt.reset_index()
px.histogram(rating_cnt,x='Rating',marginal='box',nbins=15,width=750,height=500)

##Feature Engineering

In [43]:
# Create a new DataFrame with average rating of each title and total no. of ratings
new_data = pd.DataFrame(data.groupby("Title").mean()["Rating"])
new_data['Num_of_ratings'] = pd.DataFrame(data.groupby('Title').count()['Rating'])
new_data.head()

,Rating,Num_of_ratings
Title,,
"A Light in the Storm: The Civil War Diary of Amelia Martin, Fenwick Island, Delaware, 1861 (Dear America)",0.0,1
Beyond IBM: Leadership Marketing and Finance for the 1990s,0.0,1
It Takes Two,0.0,1
"Little Comic Shop of Horrors (Give Yourself Goosebumps, Book 17)",0.0,1
Murder of a Sleeping Beauty (Scumble River Mysteries (Paperback)),0.0,1


In [14]:
# Create a pivot table with the DataFrame
book_matrix = data.pivot_table(index="user_id", columns="Title", values="Rating")
book_matrix.head()

Title,"A Light in the Storm: The Civil War Diary of Amelia Martin, Fenwick Island, Delaware, 1861 (Dear America)",Beyond IBM: Leadership Marketing and Finance for the 1990s,It Takes Two,"Little Comic Shop of Horrors (Give Yourself Goosebumps, Book 17)",Murder of a Sleeping Beauty (Scumble River Mysteries (Paperback)),"Q-Space (Star Trek The Next Generation, Book 47)",Tales of Terror and Suspense,$oft Money: The True Power in Our Nation's Capital,"' Sie belieben wohl zu scherzen, Mr. Feynman.'. Abenteuer eines neugierigen Physikers.",' Small g'. Eine Sommeridylle.,'48,'Eine gute Frau hat keinen Kopf'. EuropÃ?Â¤ische SprichwÃ?Â¶rter Ã?Â¼ber Frauen.,'O Au No Keia: Voices from Hawai'I's Mahu and Transgender Communities,'Romance of Paula Vaughn','Salem's Lot,"'Til There Was You (Special Edition, No 576)",(George),"--Olivetti, Moulinex, Chaffoteaux et Maury (Quaderns crema. Narrativa)",...AND THE HORSE HE RODE IN ON : THE PEOPLE V. KENNETH STARR,".Hack: //Legend of the Twilight, Vol. 1",".Hack: //Legend of the Twilight, Vol. 2",".NET Framework Essentials, 3rd Edition",01-01-00: A Novel of the Millennium,01-01-00: The Novel of the Millennium,"1, 2, 3: NURSERY BOARD BOOKS (Nursery Board Books)","1,000 Places to See Before You Die","1,001 Things to Do When There's Nothing to Do","1,001 Ways to Save the Planet","1,003 Great Things About America","1,003 Great Things About Getting Older","1,003 Great Things About Kids","1,401 More Things That P*Ss Me Off",10 Commandments Of Dating,10 Keys for Unlocking the Bible: Treasures That Will Change Your Life,10 Minute Guide to HTML 3.2,10 Minute Guide to Lotus Notes Mail 4.6 (Ten Minute Guide to...),10 Minutes Till Bedtime,10 Natural Laws of Successful Time and Life Management,10 Secrets for Success and Inner Peace,10 critical keys for highly effective Mormon families,...,ZÃ?Â©ro tuÃ?Â©s,"\ Lamb to the Slaughter and Other Stories (Penguin 60s S.)""","\A\"" is for Alibi : A Kinsey Millhone Mystery (A Kinsey Millhone Mystery)""","\And So It Goes\"": Adventures in Television/10113""","\Even Monkeys Fall from Trees\"" and Other Japanese Proverbs""","\God's Grandeur\"" and Other Poems (Dover Thrift Editions)""","\I Remember Laura\"": Laura Ingalls Wilder""","\I Won't Learn from You\"": And Other Thoughts on Creative Maladjustment""","\More More More,\"" Said the Baby""","\O\"" Is for Outlaw""","\Once More, With Feeling\"" : The Script Book""","\Slick Willie\"": Why America Cannot Trust Bill Clinton""","\Surely You're Joking, Mr. Feynman!\"": Adventures of a Curious Character""","\The Happy Prince\"" and Other Stories (Penguin Popular Classics)""","\The Times\"" Second Tournament of the Mind""","\The Times\"" Tournament of the Mind""","\The Twilight of the Idols (Classics S.)""","\Toward full communion\"" and \""Concordat of agreement\""""","\Treasure Island\"": According to Spike Milligan""","\Well, there's your problem\"": Cartoons""","\What Do You Care What Other People Think?\"": Further Adventures of a Curious Character""","aol.com: How Steve Case Beat Bill Gates, Nailed the Netheads, and Made Millions in the War for the Web","buongiorno!, Tl.1, Lehrbuch, Italienisch fÃ?Â¼r AnfÃ?Â¤nger",de Amor y de Sombra - Bolsillo,de Parte de La Princesa Muerta,e,eCFO: Sustaining Value in The New Corporation,eXistenZ: A Novelization,iI Paradiso Degli Orchi,"metamorfosis, La (ClÃ¡sicos selecciÃ³n series)",seaQuest 2,stardust,the Heiress Bride (sister brides),"wet sand, raven tracks",why I'm like this : True Stories,"Â¡Corre, perro, corre!",Â¿QuiÃ©n se ha llevado mi queso?,"Â¿QuÃ© me quieres, amor?",Ã?Â?ber die Freiheit.,Ã?Â?bermorgen.
user_id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Na

In [49]:
# Recommending top 10 books correlated with 'The Da Vinci Code'
da_vinci = pd.DataFrame(book_matrix.corrwith(book_matrix['The Da Vinci Code']),columns=['Correlation'])
da_vinci.dropna(inplace=True)
recom = da_vinci.join(new_data['Num_of_ratings'])
recom = recom.reset_index()
recom[recom.Num_of_ratings>40].sort_values(by=['Correlation','Num_of_ratings'],ascending=False).head(10)

,Title,Correlation,Num_of_ratings
1306,The Da Vinci Code,1.000000,76
1261,The Brethren,1.000000,47
1606,The Summons,0.845626,42
1286,The Client,0.739153,51
1519,The Pelican Brief,0.702524,60
1498,The Nanny Diaries: A Novel,0.633792,70
605,Harry Potter and the Sorcerer's Stone (Harry P...,0.575758,53
1692,Timeline,0.519361,44
640,House of Sand and Fog,0.505445,53
1619,The Testament,0.495832,52
